In [1]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)
import lightgbm as lgbm

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

/kaggle/input/optiver-trading-at-the-close/public_timeseries_testing_util.py
/kaggle/input/optiver-trading-at-the-close/train.csv
/kaggle/input/optiver-trading-at-the-close/example_test_files/sample_submission.csv
/kaggle/input/optiver-trading-at-the-close/example_test_files/revealed_targets.csv
/kaggle/input/optiver-trading-at-the-close/example_test_files/test.csv
/kaggle/input/optiver-trading-at-the-close/optiver2023/competition.cpython-310-x86_64-linux-gnu.so
/kaggle/input/optiver-trading-at-the-close/optiver2023/__init__.py


In [2]:
df_train = pd.read_csv('../input/optiver-trading-at-the-close/train.csv')

In [3]:
df_train

,stock_id,date_id,seconds_in_bucket,imbalance_size,imbalance_buy_sell_flag,reference_price,matched_size,far_price,near_price,bid_price,bid_size,ask_price,ask_size,wap,target,time_id,row_id
0,0,0,0,3180602.69,1,0.999812,13380276.64,NaN,NaN,0.999812,60651.50,1.000026,8493.03,1.000000,-3.029704,0,0_0_0
1,1,0,0,166603.91,-1,0.999896,1642214.25,NaN,NaN,0.999896,3233.04,1.000660,20605.09,1.000000,-5.519986,0,0_0_1
2,2,0,0,302879.87,-1,0.999561,1819368.03,NaN,NaN,0.999403,37956.00,1.000298,18995.00,1.000000,-8.389950,0,0_0_2
3,3,0,0,11917682.27,-1,1.000171,18389745.62,NaN,NaN,0.999999,2324.90,1.000214,479032.40,1.000000,-4.010200,0,0_0_3
4,4,0,0,447549.96,-1,0.999532,17860614.95,NaN,NaN,0.999394,16485.54,1.000016,434.10,1.000000,-7.349849,0,0_0_4
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
5237975,195,480,540,2440722.89,-1,1.000317,28280361.74,0.999734,0.999734,1.000317,32257.04,1.000434,319862.40,1.000328,2.310276,26454,480_540_195
5237976,196,480,540,349510.47,-1,1.000643,9187699.11,1.000129,1.000386,1.000643,205108.40,1.000900,93393.07,1.000819,-8.220077,26454,480_540_196
5237977,197,480,540,0.00,0,0.995789,12725436.10,0.995789,0.995789,0.995789,16790.66,0.995883,180038.32,0.995797,1.169443,26454,480_540_197
5237978,198,480,540,1000898.84,1,0.999210,94773271.05,0.999210,0.999210,0.998970,125631.72,0.999210,669893.00,0.999008,-1.540184,26454,480_540_198


In [4]:
def feature_cols(df) :
    cols = ['bid_size', 
            'ask_size', 
            'bid_price', 
            'ask_price',
            'wap',
            'imbalance_buy_sell_flag', 
            'imbalance_size', 
            'matched_size', 
            'reference_price',
            'target']
    df = df[cols]    
    return df

In [5]:
df_train.fillna(0, inplace = True)
x_train = feature_cols(df_train).drop(columns='target')
y_train = df_train['target']

In [6]:
lgbm_model = lgbm.LGBMRegressor(objective='mae', n_estimators=500, random_state=1234)
lgbm_model.fit(x_train, y_train)

LGBMRegressor(n_estimators=500, objective='mae', random_state=1234)

In [7]:
lgbm_model.booster_.save_model('lgbm_model.txt')

In [8]:
bid_size = df_train['bid_size'][5]
ask_size = df_train['ask_size'][5]
bid_price = df_train['bid_price'][5]
ask_price = df_train['ask_price'][5]
wap = df_train['wap'][5]
imbalance_buy_sell_flag = df_train['imbalance_buy_sell_flag'][5]
imbalance_size = df_train['imbalance_size'][5]
matched_size = df_train['matched_size'][5]
reference_price = df_train['reference_price'][5]
prediction_array = np.array([bid_size, 
                             ask_size, 
                             bid_price, 
                             ask_price, 
                             wap,
                             imbalance_buy_sell_flag, 
                             imbalance_size, 
                             matched_size, 
                             reference_price]).reshape(1, -1)
target = lgbm_model.predict(prediction_array)[0]
print(target)
print(df_train['target'][5])

2.3848550962307034
6.7794323


In [9]:
df_test = feature_cols(df_train).iloc[0: 2000]
df_test

,bid_size,ask_size,bid_price,ask_price,wap,imbalance_buy_sell_flag,imbalance_size,matched_size,reference_price,target
0,60651.50,8493.03,0.999812,1.000026,1.000000,1,3180602.69,13380276.64,0.999812,-3.029704
1,3233.04,20605.09,0.999896,1.000660,1.000000,-1,166603.91,1642214.25,0.999896,-5.519986
2,37956.00,18995.00,0.999403,1.000298,1.000000,-1,302879.87,1819368.03,0.999561,-8.389950
3,2324.90,479032.40,0.999999,1.000214,1.000000,-1,11917682.27,18389745.62,1.000171,-4.010200
4,16485.54,434.10,0.999394,1.000016,1.000000,-1,447549.96,17860614.95,0.999532,-7.349849
...,...,...,...,...,...,...,...,...,...,...
1995,22636.68,8748.88,1.000005,1.000336,1.000244,-1,6442673.66,28776212.43,1.000141,3.910065
1996,36414.00,4615.86,0.999113,0.999853,0.999770,0,0.00,5649183.28,0.999195,-2.819896
1997,20690.00,28981.40,1.001007,1.001539,1.001229,1,340008.11,3883946.75,1.001491,0.599623
1998,294.40,7364.25,0.999820,1.000398,0.999843,1,8805612.69,10324989.44,1.000364,2.479553


In [10]:
pd.to_pickle(df_test,'df_test.pkl')

In [11]:
X_test = df_test.drop(columns='target').iloc[15:20]
print(lgbm_model.predict(X_test))
print(df_test['target'].iloc[15:20])

[-0.31186656 -4.68565393  1.80886319 -0.29036088 -0.58475963]
15     1.720190
16    -8.170009
17    -3.589988
18   -11.439919
19     4.299879
Name: target, dtype: float64
